# Compare atmospheric parameters

Compare atmosphreic parameters, the one reconstructed by Spectractor, the other given by MERRA2

- author : Sylvie Dagoret-Campagne
- affiliation : LAL/IN2P3/CNRS
- date : November 26th 2018


In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
import matplotlib.pyplot as plt
%matplotlib inline
import sys
import os
from astropy.io import fits
import numpy as np
import pandas as pd

In [28]:
from astropy.io import fits

In [29]:
from scipy.stats import linregress
from scipy.stats import norm

# Config

In [30]:
#  column 0 : count number
#  column 1 : aerosol value
#  column 2 : pwv value
#  column 3 : ozone value
#  column 6 : data start 
#
index_atm_count=0
index_atm_aer=1
index_atm_pwv=2
index_atm_oz=3
index_atm_ps=4
index_atm_cloud=5
index_atm_data=6

NB_atm_HEADER=6
#NB_atm_DATA=len(WL)-1

# output file

In [31]:
#output_file='MERRA2_2017_M2I1NXASM_M2T1NXAER_M2T1NXRAD_ctio_atmsim.fits'

# input files : data logbook and Merra data

In [32]:
file_merra2='MERRA2_2018_M2I1NXASM_M2T1NXAER_M2T1NXRAD_ohp_AllYear.csv'
#file_logbook_ohp='logbook_stardice_jan_july18.csv'
file_logbook_ohp='logbook_stardice_oct18.csv'

## Merra data

In [33]:
df_merra2=pd.read_csv(file_merra2,index_col=0)

In [34]:
df_merra2.index.name='time'
df_merra2.sort_index(inplace=True)

In [35]:
df_merra2.head()

,ps,pwv,ozone,TOTEXTTAU,TOTANGSTR,TOTSCATAU,TAUTOT,TAUHGH,TAUMID,TAULOW
time,,,,,,,,,,
2018-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 00:30:00,NaN,NaN,NaN,0.139384,1.205785,0.133389,63.875,0.063202,2.524414,61.296875
2018-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-01 01:30:00,NaN,NaN,NaN,0.114049,1.031252,0.109637,40.000,0.000000,13.625000,26.382812
2018-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## OHP data

In [36]:
#df_ctio=pd.read_csv(file_logbook_ctio,sep=';')
df_ohp=pd.read_csv(file_logbook_ohp)

In [37]:
df_ohp.columns

Index([u'Unnamed: 0', u'ALT', u'ASKED-ALT', u'ASKED-AZ', u'AZ', u'BZERO',
       u'DATE-OBS', u'DECDEG', u'EXPNUM', u'EXPTIME',
       ...
       u'skyvar', u'subdir', u'sx', u'sy', u'x', u'x_gaussian', u'xpeak', u'y',
       u'y_gaussian', u'ypeak'],
      dtype='object', length=105)

In [38]:
#df_ctio=df_ctio.reindex(columns=[
#    'date','subdir','index','object','filter','disperser','airmass','exposure','focus','seeing',
#    'P','T','RH','W','file','target','targetx','targety','D2CCD','pixshift','thetafit','thetaint',
#    'A1','A2','ozone','PWV','VAOD','reso','D2CCD_fit,alpha_fit']).set_index('date').sort_index()

In [39]:
df_ohp.head()

,Unnamed: 0,ALT,ASKED-ALT,ASKED-AZ,AZ,BZERO,DATE-OBS,DECDEG,EXPNUM,EXPTIME,...,skyvar,subdir,sx,sy,x,x_gaussian,xpeak,y,y_gaussian,ypeak
0,0,45.864010,57.895736,275.854757,284.696327,32768.0,2018-10-22T20:09:10.955(UTC),38.784444,9828,1.0,...,61.451078,2018-10-22,0.0,0.0,651.263283,0.0,0.0,-0.000000,0.0,0.0
1,1,45.849677,57.895736,275.854757,284.672067,32768.0,2018-10-22T20:09:10.955(UTC),38.784444,9828,1.0,...,61.451078,2018-10-22,0.0,0.0,607.680114,0.0,0.0,4.831447,0.0,0.0
2,2,45.772861,57.895736,275.854757,284.514754,32768.0,2018-10-22T20:09:10.955(UTC),38.784444,9828,1.0,...,61.451078,2018-10-22,0.0,0.0,342.593705,0.0,0.0,8.967813,0.0,0.0
3,3,45.719101,57.895736,275.854757,284.409268,32768.0,2018-10-22T20:09:10.955(UTC),38.784444,9828,1.0,...,61.451078,2018-10-22,0.0,0.0,162.052152,0.0,0.0,14.812008,0.0,0.0
4,4,45.737361,57.895736,275.854757,284.463582,32768.0,2018-10-22T20:09:10.955(UTC),38.784444,9828,1.0,...,61.451078,2018-10-22,0.0,0.0,244.603954,0.0,0.0,27.340368,0.0,0.0


In [40]:
df_ohp.tail()

,Unnamed: 0,ALT,ASKED-ALT,ASKED-AZ,AZ,BZERO,DATE-OBS,DECDEG,EXPNUM,EXPTIME,...,skyvar,subdir,sx,sy,x,x_gaussian,xpeak,y,y_gaussian,ypeak
389554,425,NaN,NaN,NaN,NaN,32768.0,2018-10-26T03:47:04.645(UTC),33.7675,14215,2.0,...,233.879413,2018-10-25,0.0,0.0,436.737223,0.0,0.0,220.316165,0.0,0.0
389555,426,NaN,NaN,NaN,NaN,32768.0,2018-10-26T03:47:46.881(UTC),33.7675,14217,2.0,...,220.185779,2018-10-25,0.0,0.0,447.760298,0.0,0.0,209.034942,0.0,0.0
389556,427,NaN,NaN,NaN,NaN,32768.0,2018-10-26T03:48:29.162(UTC),33.7675,14219,2.0,...,230.557793,2018-10-25,0.0,0.0,358.513521,0.0,0.0,181.208385,0.0,0.0
389557,428,NaN,NaN,NaN,NaN,32768.0,2018-10-26T03:49:53.750(UTC),33.7675,14223,2.0,...,182.625216,2018-10-25,0.0,0.0,422.062322,0.0,0.0,247.938464,0.0,0.0
389558,429,NaN,NaN,NaN,NaN,32768.0,2018-10-26T03:50:35.994(UTC),33.7675,14225,2.0,...,243.447907,2018-10-25,0.0,0.0,405.844687,0.0,0.0,147.702694,0.0,0.0


# Select a night in OHP logbook

- The subdir name is used to get one night
- The disperser == Ron400 

In [41]:
selected_night="2018-10-22"
#selected_disperser="Ron400"

In [42]:
#selection_tag=selected_night+' & '+selected_disperser+':'

## Select the night

In [43]:
df_ohp_select=df_ohp.loc[df_ohp["subdir"]==selected_night]

In [44]:
df_ohp_select

,Unnamed: 0,ALT,ASKED-ALT,ASKED-AZ,AZ,BZERO,DATE-OBS,DECDEG,EXPNUM,EXPTIME,...,skyvar,subdir,sx,sy,x,x_gaussian,xpeak,y,y_gaussian,ypeak
0,0,45.864010,57.895736,275.854757,284.696327,32768.0,2018-10-22T20:09:10.955(UTC),38.784444,9828,1.0,...,61.451078,2018-10-22,0.0,0.0,651.263283,0.0,0.0,-0.000000,0.0,0.0
1,1,45.849677,57.895736,275.854757,284.672067,32768.0,2018-10-22T20:09:10.955(UTC),38.784444,9828,1.0,...,61.451078,2018-10-22,0.0,0.0,607.680114,0.0,0.0,4.831447,0.0,0.0
2,2,45.772861,57.895736,275.854757,284.514754,32768.0,2018-10-22T20:09:10.955(UTC),38.784444,9828,1.0,...,61.451078,2018-10-22,0.0,0.0,342.593705,0.0,0.0,8.967813,0.0,0.0
3,3,45.719101,57.895736,275.854757,284.409268,32768.0,2018-10-22T20:09:10.955(UTC),38.784444,9828,1.0,...,61.451078,2018-10-22,0.0,0.0,162.052152,0.0,0.0,14.812008,0.0,0.0
4,4,45.737361,57.895736,275.854757,284.463582,32768.0,2018-10-22T20:09:10.955(UTC),38.784444,9828,1.0,...,61.451078,2018-10-22,0.0,0.0,244.603954,0.0,0.0,27.340368,0.0,0.0
5,5,45.780299,57.895736,275.854757,284.576908,32768.0,2018-10-22T20:09:10.955(UTC),38.784444,9828,1.0,...,61.451078,2018-10-22,0.0,0.0,422.000071,0.0,0.0,45.108892,0.0,0.0
6,6,45.815727,57.895736,275.854757,284.652942,32768.0,2018-10-22T20:09:10.955(UTC),38.784444,9828,1.0,...,61.451078,2018-10-22,0.0,0.0,548.278962,0.0,0.0,45.879105,0.0,0.0
7,7,45.869249,57.895736,275.854757,284.776487,32768.0,2018-10-22T20:09:10.955(UTC),38.784444,9828,1.0,...,61.451078,2018-10-22,0.0,0.0,748.711848,0.0,0.0,53.207221,0.0,0.0
8,8,45.701540,57.895736,275.854757,284.436152,32768.0,2018-10-22T20:09:10.955(UTC),38.784444,9828,1.0,...,61.451078,2018-10-22,0.0,0.0,173.367543,0.0,0.0,64.663036,0.0,0.0
9,9,45.802798,57.895736,275.854757,284.672777,32768.0,2018-10-22T20:09:10.955(UTC),38.784444,9828,1.0,...,61.451078,2018-10-22,0.0,0.0,556.653633,0.0,0.0,82.469882,0.0,0.0


In [45]:
len(df_ohp_select)

18623

## Select the disperser

In [46]:
#df_ctio_select_sel=df_ctio_select[df_ctio_select["disperser"]==selected_disperser]

In [47]:
#len(df_ctio_select_sel)

In [48]:
#df_ctio_select_sel.head()

# Find the $time_{start}$ and $time_{stop}$ at CTIO during this night

In [49]:
all_ohp_datetime=pd.to_datetime(df_ohp_select.index.get_values())

In [50]:
ohp_start_time=all_ohp_datetime[0]
ohp_stop_time=all_ohp_datetime[-1]
print 'OHP :', ohp_start_time,' --> ', ohp_stop_time

OHP : 1970-01-01 00:00:00  -->  1970-01-01 00:00:00.000018622


In [51]:
df_merra2.index.get_values()

array(['2018-01-01 00:00:00', '2018-01-01 00:30:00',
       '2018-01-01 01:00:00', ..., '2018-10-31 22:30:00',
       '2018-10-31 23:00:00', '2018-10-31 23:30:00'], dtype=object)

In [52]:
# convert the string into timestamp
#-------------------------------------
all_datetime=pd.to_datetime(df_merra2.index.get_values())

In [53]:
print len(all_datetime)

14592


In [55]:
idx=0
for timestamp in all_datetime:
    month=timestamp.month
    day=timestamp.days_in_month
    hour=timestamp.hour
    minu=timestamp.minute
    
    # goal take every hours pressure, PWV and Ozone, and average the aerosols and clouds 30 min before and 30 min after
    if minu==0:
        ps=df_merra2.iloc[idx]["ps"]/100. # in hecto-Pa
        pwv=df_merra2.iloc[idx]["pwv"]
        ozone=df_merra2.iloc[idx]["ozone"]
       
        deltat=(all_datetime-timestamp).total_seconds()
        idx_before=np.where(deltat==-1800)[0] # which is the index 1800 seconds before
        idx_after=np.where(deltat==1800)[0]   # which is the index 1800 seconds after
        if idx_before.size==0 and idx_after.size==1:
            aer=df_merra2.iloc[idx_after[0]]["TOTEXTTAU"]
            clouds=df_merra2.iloc[idx_after[0]]["TAUTOT"]
        elif idx_before.size==1 and idx_after.size==0:
            aer=df_merra2.iloc[idx_before[0]]["TOTEXTTAU"] 
            clouds=df_merra2.iloc[idx_before[0]]["TAUTOT"] 
        elif idx_before.size==1 and idx_after.size==1:
            aer=np.mean([df_merra2.iloc[idx_before[0]]["TOTEXTTAU"] ,df_merra2.iloc[idx_after[0]]["TOTEXTTAU"] ])
            clouds=np.mean([df_merra2.iloc[idx_before[0]]["TAUTOT"] ,df_merra2.iloc[idx_after[0]]["TAUTOT"] ])
        else:
            aer=0
            clouds=0
        
        if timestamp >= ohp_start_time and timestamp <= ohp_stop_time:
        
            print 'idx=',idx, ' timestamp=',timestamp,' P=',ps,' pwv=',pwv,' ozone =',ozone
            print ' \t aer=',aer, ' , clouds = ',clouds,' , : index before and after :', idx_before,' ',idx_after, idx_before.size, idx_after.size
        
    idx+=1  # increase the counter

# Find the corresponding atmospheric parameters in MERRA2

In [56]:
def GetAtmosphericParameters(timestamp0,df_merra2):
    """
    GetAtmosphericParameters(timestamp0,df_merra2) :
     return closest time atmospheric parameters from merra dataset at timestamp0
    
    input arg:
    - timestamp0 : timestamp of the CTIO image in pd.DateTime type
    - df_merra2  : pandas dataset holding all atmospheric parameters indexed by time
    
    where :
    "ps","pwv","ozone" are estimated every hours
    'TOTEXTTAU','TOTANGSTR','TOTSCATAU','TAUTOT','TAUHGH','TAUMID','TAULOW' are estimated ever hour at hour and a half 
    
    then  df_merra2 is split into two datasets in which there are no NA value.
    The estimated value the closest in time are returned
    
    return:
        - ps : presure in hPa from dataset1
        - pwv : Precipitable Water vapour in mm from dataset1
        - ozone : Ozone in Dobson Unit in dataset1 
        - aer   : vertical aerosol optical depth at 550 nm in dataset2
        - clouds : clouds in vertical depth
        - deltat1,deltat2  : time delay wrt timestamp
    
    """
    
    # decode the time form timestamp
    year0=timestamp0.year
    month0=timestamp0.month
    day0=timestamp0.days_in_month
    hour0=timestamp0.hour
    minu0=timestamp0.minute
    
    # remove NA in rows and split in 2 datasets, dataset1,dataset2 without NA 
    # datset1 with ps, pwv ozone 
    dataset1_m2=df_merra2.dropna(axis=0,how='all',subset=["ps","pwv","ozone"]).loc[:, 'ps':'ozone']
    # datset2 with aerosols and clouds
    dataset2_m2=df_merra2.dropna(axis=0,how='all',subset=['TOTEXTTAU','TOTANGSTR','TOTSCATAU','TAUTOT','TAUHGH','TAUMID','TAULOW']).loc[:,'TOTEXTTAU':'TAULOW']
    
    
    # convert the string into timestamp
    #-------------------------------------
    all_datetime1_m2=pd.to_datetime(dataset1_m2.index.get_values())
    all_datetime2_m2=pd.to_datetime(dataset2_m2.index.get_values())
    
   
    # get time difference between timestamp = timestamp-merra2-timestamp-ctio
    #---------------------------------------
    deltat1=(all_datetime1_m2-timestamp0).total_seconds()
    deltat2=(all_datetime2_m2-timestamp0).total_seconds()
    
    delays1=np.abs(deltat1)
    idx1=np.where(delays1==delays1.min())[0][0]
    
    delays2=np.abs(deltat2)
    idx2=np.where(delays2==delays2.min())[0][0]
    
    if False:
        print 'deltat1 :' ,deltat1[idx1], pd.Timedelta(deltat1[idx1],unit='s')
        print dataset1_m2.iloc[idx1,:]
        
        print 'deltat2 :' ,deltat2[idx2],pd.Timedelta(deltat2[idx2],unit='s')
        print dataset2_m2.iloc[idx2,:]
        
    ps=dataset1_m2.iloc[idx1]["ps"]/100. # convert Pa into hecto-Pa
    pwv=dataset1_m2.iloc[idx1]["pwv"]
    ozone=dataset1_m2.iloc[idx1]["ozone"]
    aer=dataset2_m2.iloc[idx2]["TOTEXTTAU"]
    clouds=dataset2_m2.iloc[idx2]["TAUTOT"]
    
    if False:
        print 'idx1=',idx1,' P=',ps,' pwv=',pwv,' ozone =',ozone
        print 'idx2=',idx2,' aer=',aer, ' , clouds = ',clouds
    
    return ps,pwv,ozone,aer,clouds,deltat1[idx1]/60.,deltat2[idx2]/60.                          

## loop on selected OHP dates to find Merra2 atmospheric parameters

In [58]:
merra2_ps=[]
merra2_pwv=[]
merra2_ozone=[]
merra2_aer=[]
merra2_clouds=[]

for theohpdate in all_ohp_datetime:
    ps,pwv,ozone,aer,clouds,deltat1,deltat2=GetAtmosphericParameters(theohpdate,df_merra2)
    merra2_ps.append(ps)
    merra2_pwv.append(pwv)
    merra2_ozone.append(ozone)
    merra2_aer.append(aer)
    merra2_clouds.append(clouds)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
merra2_ps=np.array(merra2_ps)
merra2_pwv=np.array(merra2_pwv)
merra2_ozone=np.array(merra2_ozone)
merra2_aer=np.array(merra2_aer)
merra2_clouds=np.array(merra2_clouds)

# Build the augmented OHP logbook for that night

In [ ]:
df_ohp_new=df_ohp_select

In [ ]:
df_ohp_new["m2_ps"]=merra2_ps
df_ohp_new["m2_pwv"]=merra2_pwv
df_ohp_new["m2_ozone"]=merra2_ozone
df_ohp_new["m2_aer"]=merra2_aer
df_ohp_new["m2_clouds"]=merra2_clouds

In [ ]:
df_ohp_new

In [ ]:
geedfsgf

# Do the analysis

In [ ]:
def StudyCorrelation(df,var1,var2,title,xtitle,ytitle,deg=1,FitFlag=True):
    theX=df[var1].values
    theY=df[var2].values
    
    slope,intercept,rvalue,pvalue,stderr=linregress(theX,theY)
    nbsig=norm.ppf(1-pvalue)
    
    statinfo="r={:4.3f}, pvalue = {:3.2g}, $\sigma$ = {:2.1f}".format(rvalue,pvalue,nbsig)
    
    fulltitle=title+statinfo
    
    if FitFlag:
        theZ,theCov=np.polyfit(theX,theY,deg,cov=True)
        pol = np.poly1d(theZ)
        Xfit=np.linspace(theX.min(),theX.max(),50)
        Yfit=pol(Xfit)
    
    
    ax=df.plot.scatter(x=var1,y=var2,c='DarkBlue',grid=True,title=fulltitle,figsize=(10, 6),fontsize = 16)
    
    if FitFlag:
        ax.plot(Xfit,Yfit,'r-')
    ax.set(xlabel=xtitle, ylabel=ytitle)
    plt.show()
    
    corr=df.corr()
    print corr
    
    plt.figure(figsize=(4, 3))
    plt.imshow(corr, cmap='bwr', interpolation='none', aspect='auto',vmin=-1,vmax=+1)
    plt.colorbar()
    plt.xticks(range(len(corr)), corr.columns, rotation='vertical')
    plt.yticks(range(len(corr)), corr.columns);
    plt.suptitle('Correlation Matrix ', fontsize=15, fontweight='bold')
    plt.show()
    

## Pressure

In [ ]:
df_ps=df_ctio_new[["P","m2_ps"]]

In [ ]:
title="CTIO::"+selection_tag+"Pressure P(CTIO) vs P(MERRA) "

In [ ]:
StudyCorrelation(df_ps,"m2_ps","P",title,"Pressure from Merra (hPa)","Pressure from CTIO (hPa)")

## PWV

In [ ]:
df_pwv=df_ctio_new[["PWV","m2_pwv"]]

In [ ]:
title="CTIO::"+selection_tag+"PWV : pwv(CTIO) vs pwv(MERRA) "

In [ ]:
df_pwv.head()

In [ ]:
StudyCorrelation(df_pwv,"m2_pwv","PWV",title,"PWV from MERRA (mm)","PWV at CTIO (mm)")

## Ozone

In [ ]:
df_ozone=df_ctio_new[["ozone","m2_ozone"]]

In [ ]:
title="CTIO::"+selection_tag+"Ozone : O3(CTIO) vs O3(MERRA) "

In [ ]:
StudyCorrelation(df_ozone,"m2_ozone","ozone",title,"O3 from MERRA (DbU)","O3 at CTIO (DbU)")

## VAOD

In [ ]:
df_aer=df_ctio_new[["VAOD","m2_aer"]]

In [ ]:
title="CTIO::"+selection_tag+"Aerosols : VAOD(CTIO) vs VAOD(MERRA) "

In [ ]:
StudyCorrelation(df_aer,"m2_aer","VAOD",title,"VAOD from MERRA","VAOD at CTIO")